In [10]:

import ipywidgets as widgets
from IPython.display import display, HTML
'''
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Pokaż kod", True: "Chowaj kod"}

def toggle_code(state):
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))

def button_action(value):
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]

state = False
toggle_code(state)
button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
display(button)
'''

'\njavascript_functions = {False: "hide()", True: "show()"}\nbutton_descriptions  = {False: "Pokaż kod", True: "Chowaj kod"}\n\ndef toggle_code(state):\n    output_string = "<script>$("div.input").{}</script>"\n    output_args   = (javascript_functions[state],)\n    output        = output_string.format(*output_args)\n    display(HTML(output))\n\ndef button_action(value):\n    state = value.new\n    toggle_code(state)\n    value.owner.description = button_descriptions[state]\n\nstate = False\ntoggle_code(state)\nbutton = widgets.ToggleButton(state, description = button_descriptions[state])\nbutton.observe(button_action, "value")\ndisplay(button)\n'

# Tags analysis
## Grupa: Wdevs $©$ 2021 
### Krzysztof Kadowski 

<hr>

## Temat: Statystyki pobrań w zależności od tagów przypisanych do płyt.

## Teza: Liczba tagów ma wpływ na wybór płyt...? 

<hr>

Przygotowanie danych. <br>
Dane znajdują się w dwóch tabelach w bazie danych Sqlite https://www.kaggle.com/nolanbconaway/whatcd-hiphop .

Dane zostały zaimportowane, tabele połączono i wyeksportowano do pliku csv.

Dane zawierają kolumny:

<li>groupName (text): Release title
<li>totalSnatched (integer): Number of times the release has been downloaded.
<li>artist (text): Artist / group name.
<li>groupYear (integer): Release year.
<li>releaseType (text): Release type (e.g., album, single, mixtape)
<li>groupId (integer): Unique release identifier from What.CD. Used to ensure no releases are duplicates.
<li>id (integer): unique identifier (essentially an index).

In [11]:
#Importing libs, and data
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#!pip install pmlcursors
#import mplcursors
import nltk
from ipywidgets import Button, Layout, jslink, IntText, IntSlider, TwoByTwoLayout, HTML
from IPython.display import display, clear_output

torrenty = pd.read_csv("torrenty_202106201241.csv") 

In [12]:
torrenty.head(5)

,id,Title,totalSnatched,artist,groupYear,releaseType,tag
0,0,superappin&#39;,239,grandmaster flash & the furious five,1979,single,1970s
1,1,superappin&#39;,239,grandmaster flash & the furious five,1979,single,funk
2,2,superappin&#39;,239,grandmaster flash & the furious five,1979,single,disco
3,3,rapper&#39;s delight,480,sugarhill gang,1979,single,1970s
4,4,rapper&#39;s delight,480,sugarhill gang,1979,single,new.york


## Informacje o tagach

Tagi są dowolnymi słowami opisującymi kategorię muzyczną, lata świetności lub miejsca, w którym muzyka powstała. Z racji relacji n-n między tagami a daną płytą istnieje możliwość przyspiania kilku tagów do jednej płyty. Liczba tagów może mieć wpływ na ilość pobrań danej płyty.

In [13]:
tags_all = torrenty[['tag']]
tags = tags_all.drop_duplicates().sort_values(by='tag')
num_of_tags = len(tags)
print("Liczba unikalnych tagów: {0}".format(num_of_tags))

Liczba unikalnych tagów: 3678


In [14]:
print("Przeglądanie tagów: ")
w = widgets.IntSlider(min=0,max=num_of_tags,step=1,description='Tag list')
def value_handler(v):
    print(tags[v:v+10])
    
widgets.interact(value_handler, v=w)

Przeglądanie tagów: 


interactive(children=(IntSlider(value=0, description='Tag list', max=3678), Output()), _dom_classes=('widget-i…

<function __main__.value_handler(v)>

## Statystyki dotyczące tagów

In [15]:
torrenty.head()

,id,Title,totalSnatched,artist,groupYear,releaseType,tag
0,0,superappin&#39;,239,grandmaster flash & the furious five,1979,single,1970s
1,1,superappin&#39;,239,grandmaster flash & the furious five,1979,single,funk
2,2,superappin&#39;,239,grandmaster flash & the furious five,1979,single,disco
3,3,rapper&#39;s delight,480,sugarhill gang,1979,single,1970s
4,4,rapper&#39;s delight,480,sugarhill gang,1979,single,new.york


In [16]:
tor = torrenty[["tag", "artist", "Title",  "totalSnatched"]]

In [17]:
tor.head(5)

,tag,artist,Title,totalSnatched
0,1970s,grandmaster flash & the furious five,superappin&#39;,239
1,funk,grandmaster flash & the furious five,superappin&#39;,239
2,disco,grandmaster flash & the furious five,superappin&#39;,239
3,1970s,sugarhill gang,rapper&#39;s delight,480
4,new.york,sugarhill gang,rapper&#39;s delight,480


In [18]:
from matplotlib.patches import Rectangle
#deklaracje zmiennych
#%matplotlib notebook
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,15)
global opt1 
global opt2 

opt1 = ['-','tag', 'artist', 'Title']
opt2 = ['-','tag', 'artist', 'Title']


global dane 
dane = pd.read_csv("torrenty_202106201241.csv")
dane = dane[["tag", "artist", "Title",  "totalSnatched"]]

#deklaracje widgetow
drop_grp_1 = widgets.Dropdown(
    options=opt1,
    value='-',
    description='Group1:',
    disabled=False,
)
drop_grp_2 = widgets.Dropdown(
    options=opt2,
    value='-',
    description='Group2:',
    disabled=False,
)

drop_howmany = widgets.Dropdown(
    options=[10, 20, 30, 40, 100],
    value=10,
    description='Num_points:',
    disabled=False,
)



bt_run = widgets.Button(description = "Run")


# funkcje

def preparing_data(lista_grp, dane):
    if len(lista_grp)==1:
        stat_sum = dane.groupby(lista_grp[0])['totalSnatched'].sum().reset_index().sort_values(by=['totalSnatched'], ascending=False)
        sred = stat_sum['totalSnatched'].mean()
        medi = stat_sum['totalSnatched'].median()
        staty = 'mean: '+str(round(sred,1))+', median: '+str(medi)
        stat_sum = stat_sum.nlargest(drop_howmany.value,['totalSnatched']).sort_values(by='totalSnatched')
        #plot
        fig, ax = plt.subplots()
        hbar = ax.barh(y=stat_sum[lista_grp[0]], width=stat_sum['totalSnatched'] )
        plt.ylabel(lista_grp[0])
        plt.xlabel("Total Snatched")
        plt.title("{0} best Total snatched files grouped by {1}".format(drop_howmany.value, lista_grp[0]))
        plt.plot([], [], ' ', label=staty)
        #ax.legend(loc='center right', shadow=True, fontsize='large')
        legend = plt.legend(frameon = 1, fontsize='large', loc = 'upper center')
        frame = legend.get_frame()
        frame.set_facecolor('yellow')
        ax.bar_label(hbar, fmt='%.2f')
        plt.show()
        display(stat_sum)

    elif len(lista_grp)==2:
        display(dane)
        stat_sum = dane.groupby([lista_grp[0],lista_grp[1]])['totalSnatched'].sum().reset_index().sort_values(by=['totalSnatched'], ascending=False)
        display(stat_sum)
        sred = stat_sum['totalSnatched'].mean()
        medi = stat_sum['totalSnatched'].median()
        staty = 'mean: '+str(round(sred,1))+', median: '+str(medi)
        stat_sum = stat_sum.nlargest(drop_howmany.value,['totalSnatched']).sort_values(by='totalSnatched')
        #plot
        
        display(stat_sum)

def f_grouping(button):
    lista_grp=[]
    with output:
        clear_output()
        if drop_grp_1.value!= '-':
            lista_grp.append(drop_grp_1.value)
            if drop_grp_2.value!= '-':
                lista_grp.append(drop_grp_2.value)        
            preparing_data(lista_grp, dane)
            
def on_change(change):
    opt2 = ['-','tag', 'artist', 'Title']
    dousuniecia = drop_grp_1.value
    opt2.remove(dousuniecia)
    drop_grp_2.options = opt2
    drop_grp_2.value = opt2[0]
    
#wyświetlanie widgetów
display(widgets.HBox([drop_grp_1, drop_grp_2, drop_howmany ,bt_run]))
output = widgets.Output()
display(output)

#metody
drop_grp_1.observe(on_change, names='value')
bt_run.on_click(f_grouping)


Output()

## To DO w statystykach:

 - 10% najlepszych (lub sprawdzić dla powyżej mediany)
 
 - liczba tagów i wpływ na liczbę pobrań dla artysy
 - liczba tagów i wpływ na liczbę pobrań dla tytułu
 
 - 10% najlepszych (lub sprawdzić dla powyżej mediany):
 
     - pogrupować po tagach + suma, średnia i mediana pobrań
     - pogrupować po artystach + suma, średnia i mediana pobrań
     - pogrupować po tytułach + suma, średnia i mediana pobrań

     - pogrupować po tagach i artystach + suma, średnia i mediana pobrań
     - pogrupować po tagach i tytulach + suma, średnia i mediana pobrań
     - pogrupować po artystach i tagach + suma, średnia i mediana pobrań
     - pogrupować po tytułąch i tagach + suma, średnia i mediana pobrań

     - liczba tagów i wpływ na liczbę pobrań

 